In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency

In [65]:
# loading in dataset
df = pd.read_csv("/Users/cartermain/Downloads/BankChurners.csv")

In [66]:
# getting familiar with dataset
print(df.head(3))

   CLIENTNUM     Attrition_Flag  Customer_Age Gender  Dependent_count  \
0  768805383  Existing Customer            45      M                3   
1  818770008  Existing Customer            49      F                5   
2  713982108  Existing Customer            51      M                3   

  Education_Level Marital_Status Income_Category Card_Category  \
0     High School        Married     $60K - $80K          Blue   
1        Graduate         Single  Less than $40K          Blue   
2        Graduate        Married    $80K - $120K          Blue   

   Months_on_book  ...  Months_Inactive_12_mon  Contacts_Count_12_mon  \
0              39  ...                       1                      3   
1              44  ...                       1                      2   
2              36  ...                       1                      0   

   Credit_Limit  Total_Revolving_Bal  Avg_Open_To_Buy  Total_Amt_Chng_Q4_Q1  \
0       12691.0                  777          11914.0                 

In [67]:
# counting null values
print(df.isnull().sum())

CLIENTNUM                   0
Attrition_Flag              0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
dtype: int64


First, we'll check the correlation each feature has with credit card churn

In [68]:
# running a for loop to turn object dtype columns into numeric columns and keeping record of column names to drop 
le = preprocessing.LabelEncoder()
droppable_columns = []
for column in df.columns:
    if df[column].dtype == "object":
        df[column + "_int"] = le.fit_transform(df[column])
        droppable_columns.append(column)
    else:
        continue

In [69]:
# creating new dataframe with int and float dtypes exclusively to run correlation
numeric_df = df.drop(columns = droppable_columns)
numeric_df = numeric_df.drop(columns = "CLIENTNUM")

In [70]:
# running correlation for churn on numeric-only dataframe with sorted values from high to low
print(numeric_df.corr()["Attrition_Flag_int"].sort_values(ascending = False))

Attrition_Flag_int          1.000000
Total_Trans_Ct              0.371403
Total_Ct_Chng_Q4_Q1         0.290054
Total_Revolving_Bal         0.263053
Avg_Utilization_Ratio       0.178410
Total_Trans_Amt             0.168598
Total_Relationship_Count    0.150005
Total_Amt_Chng_Q4_Q1        0.131063
Gender_int                  0.037272
Credit_Limit                0.023873
Card_Category_int           0.006038
Avg_Open_To_Buy             0.000285
Education_Level_int        -0.005551
Months_on_book             -0.013687
Income_Category_int        -0.017584
Customer_Age               -0.018203
Marital_Status_int         -0.018597
Dependent_count            -0.018991
Months_Inactive_12_mon     -0.152449
Contacts_Count_12_mon      -0.204491
Name: Attrition_Flag_int, dtype: float64


Nothing has an incredibly signficant correlation, let's check average for each feature grouped by churned status

In [71]:
# printing mean value for each feature for churned and existing customers
print("Churned customer averages")
print(round(numeric_df.loc[df["Attrition_Flag_int"] == 1].mean(),0))
print("")
print("Current customer averages")
print(round(numeric_df.loc[df["Attrition_Flag_int"] == 0].mean(),0))

Churned customer averages
Customer_Age                  46.0
Dependent_count                2.0
Months_on_book                36.0
Total_Relationship_Count       4.0
Months_Inactive_12_mon         2.0
Contacts_Count_12_mon          2.0
Credit_Limit                8727.0
Total_Revolving_Bal         1257.0
Avg_Open_To_Buy             7470.0
Total_Amt_Chng_Q4_Q1           1.0
Total_Trans_Amt             4655.0
Total_Trans_Ct                69.0
Total_Ct_Chng_Q4_Q1            1.0
Avg_Utilization_Ratio          0.0
Attrition_Flag_int             1.0
Gender_int                     0.0
Education_Level_int            3.0
Marital_Status_int             1.0
Income_Category_int            3.0
Card_Category_int              0.0
dtype: float64

Current customer averages
Customer_Age                  47.0
Dependent_count                2.0
Months_on_book                36.0
Total_Relationship_Count       3.0
Months_Inactive_12_mon         3.0
Contacts_Count_12_mon          3.0
Credit_Limit          

Let's find the features with the biggest difference in mean between churned and active customers

In [72]:
# adding the delta between feature means into a dataframe with sorted absolute values to check largest deltas
df_columns = []
df_deltas = []
for feature in numeric_df.columns:
    if feature == "Attrition_Flag_int" or feature == "CLIENTNUM":
        continue
    else:
        delta = (df[feature][df["Attrition_Flag_int"] == 1].mean() - df[feature][df["Attrition_Flag_int"] == 0].mean()) / df[feature][df["Attrition_Flag_int"] == 0].mean()
        df_columns.append(feature)
        df_deltas.append(round(abs(delta),2) * 100)
deltas_df = pd.DataFrame(columns = ["Delta"], index = df_columns, data = df_deltas)
print(deltas_df["Delta"].sort_values(ascending = False))

Total_Revolving_Bal         87.0
Avg_Utilization_Ratio       82.0
Total_Trans_Ct              53.0
Total_Trans_Amt             50.0
Total_Ct_Chng_Q4_Q1         34.0
Contacts_Count_12_mon       21.0
Total_Relationship_Count    19.0
Months_Inactive_12_mon      16.0
Gender_int                  12.0
Total_Amt_Chng_Q4_Q1        11.0
Card_Category_int            7.0
Credit_Limit                 7.0
Dependent_count              3.0
Marital_Status_int           2.0
Income_Category_int          2.0
Months_on_book               1.0
Education_Level_int          1.0
Customer_Age                 1.0
Avg_Open_To_Buy              0.0
Name: Delta, dtype: float64


This order makes sense given people churning will be using their card infrequently leading up to their churn. Now let's find out which of these differences we would have a chance of observing at random via Chi-Square and Two-Sample T-Tests

In [73]:
significant = []
insignificant = []
for feature in numeric_df.columns:
    if feature == "CLIENTNUM" or feature == "Attrition_Flag_int":
        continue
    elif "_int" in feature:
        #chi square
        table = pd.crosstab(df["Attrition_Flag_int"], df[feature])
        chi2, pval, dof, expected = chi2_contingency(table)
        if pval < 0.05:
            print(feature + ": significant")
            significant.append(feature)
        else:
            print(feature + ": insignificant")
            insignificant.append(feature)
    else:
        #two-sample t-test
        a = df[feature][df["Attrition_Flag_int"] == 1]
        b = df[feature][df["Attrition_Flag_int"] == 0]
        tstat, pval = ttest_ind(a, b)
        if pval < 0.05:
            print(feature + ": signficant")
            significant.append(feature)
        else:
            print(feature + ": insignificant")
            insignificant.append(feature)

Customer_Age: insignificant
Dependent_count: insignificant
Months_on_book: insignificant
Total_Relationship_Count: signficant
Months_Inactive_12_mon: signficant
Contacts_Count_12_mon: signficant
Credit_Limit: signficant
Total_Revolving_Bal: signficant
Avg_Open_To_Buy: insignificant
Total_Amt_Chng_Q4_Q1: signficant
Total_Trans_Amt: signficant
Total_Trans_Ct: signficant
Total_Ct_Chng_Q4_Q1: signficant
Avg_Utilization_Ratio: signficant
Gender_int: significant
Education_Level_int: insignificant
Marital_Status_int: insignificant
Income_Category_int: significant
Card_Category_int: insignificant


Let's find the average age at which people sign up as cardholders by subtracting their age by number of months they have been on the books

In [74]:
df["signup_age"] = df["Customer_Age"] - round(df["Months_on_book"] / 12,0)

In [75]:
print("The average age at which people open an account is " + str(round(df["signup_age"].mean(),0)))

The average age at which people open an account is 43.0


Lastly, let's find the breakdown of card category by income level

In [76]:
print(df["Income_Category_int"].groupby(df["Card_Category_int"]).mean().sort_values(ascending = False))
print("")
income_category_crosstab = pd.crosstab(df["Income_Category_int"], df["Card_Category_int"])
print(income_category_crosstab)

Card_Category_int
0    2.887134
2    2.800000
3    2.571171
1    2.387931
Name: Income_Category_int, dtype: float64

Card_Category_int       0   1  2    3
Income_Category_int                  
0                     645  18  4   60
1                    1675  15  1   99
2                    1273  29  4   96
3                    1395  21  2  117
4                    3403  24  4  130
5                    1045   9  5   53


In [77]:
chi2, pval, dof, expected = chi2_contingency(income_category_crosstab)
print(pval)

1.2106873689789959e-14


There is a signficant difference in income level and card category as exemplified by the crosstab and mean income category above